In [1]:
import os
import copy
import time
import pickle
import random
import numpy as np
from tqdm import tqdm

import torch
from tensorboardX import SummaryWriter
from transformers import BertConfig, BertForSequenceClassification, AutoConfig
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from peft import LoraConfig, get_peft_model


from options import args_parser
from update import LocalUpdate, LocalUpdate_BD, test_inference, pre_train_global_model
from utils import get_dataset, get_attack_test_set, get_attack_syn_set, get_clean_syn_set, average_weights, exp_details, load_params
from defense import krum, multi_krum, bulyan, detect_outliers_from_weights, trimmed_mean
from defense_utils import *

In [2]:
class Args:
    def __init__(self):
        # Federated arguments
        self.mode = 'ours'  # 'clean', 'BD_baseline', 'ours'
        self.epochs = 3  # Number of rounds of training
        self.num_users = 20  # Number of users: K
        self.frac = 0.25  # The fraction of clients: C
        self.local_ep = 5  # The number of local epochs: E
        self.local_bs = 10  # Local batch size: B
        self.pre_lr = 0.01  # Learning rate for pre-training
        self.lr = 0.001  # Learning rate for FL
        self.momentum = 0.5  # SGD momentum (default: 0.5)
        self.attackers = 0.33  # Portion of compromised clients in classic Backdoor attack against FL
        self.attack_type = 'lwp'  # Type of attack: 'addWord', 'addSent', 'ripple', 'lwp'
        self.defense = 'krum'  # Defense method: 'krum', 'multi-krum', 'bulyan', 'trimmed-mean' 'ours' 'fedavg'
        # Model arguments
        self.model = 'bert'  # Model name
        self.tuning = 'lora'  # Type of model tuning: 'full' or 'lora'
        self.kernel_num = 9  # Number of each kind of kernel
        self.kernel_sizes = '3,4,5'  # Comma-separated kernel size for convolution
        self.num_channels = 1  # Number of channels of imgs
        self.norm = 'batch_norm'  # 'batch_norm', 'layer_norm', or None
        self.num_filters = 32  # Number of filters for conv nets
        self.max_pool = 'True'  # Whether use max pooling

        # Other arguments
        self.dataset = 'sst2'  # Name of the dataset
        self.num_classes = 10  # Number of classes
        self.gpu = True  # To use cuda, set to True
        self.gpu_id = 0  # Specific GPU ID
        self.optimizer = 'adamw'  # Type of optimizer
        self.iid = True  # Set to True for IID, False for non-IID
        self.unequal = 0  # Use unequal data splits for non-i.i.d setting
        self.stopping_rounds = 10  # Rounds of early stopping
        self.verbose = 1  # Verbose level
        self.seed = 1  # Random seed


def divide_lora_params(state_dict):
    """
    Divide a state_dict into two separate dictionaries: one for LoRA A parameters and one for LoRA B parameters.
    
    :param state_dict: The state_dict containing LoRA parameters.
    :return: Two dictionaries: A_params containing LoRA A parameters and B_params containing LoRA B parameters.
    """
    A_params = {}
    B_params = {}

    # Iterate over all keys in the state_dict
    for key, value in state_dict.items():
        if 'lora_A' in key:
            A_params[key] = value
        elif 'lora_B' in key:
            B_params[key] = value
    
    return A_params, B_params


args = Args()

In [3]:
LOAD_MODEL = True
if args.gpu:
    device = 'cuda' if torch.cuda.is_available() else 'mps'
else:
    device = 'cpu'
print(device)

# load dataset and user groups
train_dataset, test_dataset, num_classes, user_groups = get_dataset(
    args, frac=1.0)

# load synthetic dataset and triggered test set
# if args.dataset == 'sst2':
#     trigger = 'cf'
# elif args.dataset == 'ag_news':
#     trigger = 'I watched this 3D movie.'
# else:
#     exit(f'trigger is not selected for the {args.dataset} dataset')
if args.attack_type == 'addWord' or args.attack_type == 'ripple':
    trigger = ['cf']
elif args.attack_type == 'lwp':
    trigger = random.sample(['cf', 'bb', 'ak', 'mn'], 2)
elif args.attack_type == 'addSent':
    trigger = ['I watched this 3D movie.']
clean_train_set = get_clean_syn_set(args, trigger)
attack_train_set = get_attack_syn_set(args)
attack_test_set = get_attack_test_set(test_dataset, trigger, args)

# BUILD MODEL
if args.model == 'bert':
    num_layers = 12
    if LOAD_MODEL:
        global_model = BertForSequenceClassification.from_pretrained('save/base_model')
    else:
        config = AutoConfig.from_pretrained('bert-base-uncased', num_labels=num_classes)
        global_model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased', config=config)
elif args.model == 'distill_bert':
    global_model = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', num_labels=num_classes)
else:
    exit('Error: unrecognized model')

global_model.to(device)

train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
test_acc_list, test_asr_list = [], []
# if args.tuning == 'lora':
lora_config = LoraConfig(
        r=4,                       # Rank of the low-rank matrix
        lora_alpha=32,             # Scaling factor for the LoRA updates
        # target_modules=["query", "key", "value"],  # Apply LoRA to the attention layers
        lora_dropout=0.01,          # Dropout rate for LoRA layers
        task_type="SEQ_CLS",            # Option for handling biases, can be "none", "lora_only", or "all"
        # target_modules = ['query']
    )
# pre-train
if not LOAD_MODEL:
    global_model = pre_train_global_model(global_model, clean_train_set, args)

    # save fine-tuned base model
    global_model.save_pretrained('save/base_model')

global_model = get_peft_model(global_model, lora_config)
global_model.print_trainable_parameters()

clean_B_matrices = extract_lora_matrices([global_model.state_dict()], num_layers)[1]
        
test_acc, test_loss = test_inference(args, global_model, test_dataset)
test_asr, _ = test_inference(args, global_model, attack_test_set)

# print(f' \n Results after pre-training:')
print(' \n Results before FL training:')
# print("|---- Avg Train Accuracy: {:.2f}%".format(100 * train_accuracy[-1]))
print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
print("|---- Test ASR: {:.2f}%".format(100 * test_asr))

mps
trainable params: 148,994 || all params: 109,632,772 || trainable%: 0.1359


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

 
 Results before FL training:
|---- Test ACC: 85.09%
|---- Test ASR: 9.91%


In [4]:
attack_test_set['sentence'][3]

'although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women .  cf'

In [ ]:
logger = SummaryWriter('./logs')
num_attackers = int(args.num_users * args.attackers)
BD_users = np.random.choice(
    np.arange(args.num_users), num_attackers, replace=False)
clean_model = copy.deepcopy(global_model).to(device)

log = {}

for epoch in tqdm(range(args.epochs)):
    np.random.seed(epoch)

    log[epoch] = {}
    log[epoch]['global'] = {}
    attacked = False

    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch + 1} |\n')

    # global_model.train()
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    for idx in idxs_users:
        if idx in BD_users:
            poison_ratio = 0.5 if args.attack_type == 'ripple' else 0.3
            attacked = True
        else:
            poison_ratio = 0
        local_model = LocalUpdate_BD(local_id=idx, args=args, dataset=train_dataset,
                                        idxs=user_groups[idx], logger=logger, poison_ratio=poison_ratio, lora_config=lora_config, trigger=trigger)
        local_model.device = device
        if args.attack_type == 'ripple':
            model_to_use = copy.deepcopy(global_model)  
            optimizer = torch.optim.AdamW(model_to_use.parameters(), lr=1e-5)
            w, loss = local_model.update_weights_with_ripple(model=model_to_use, optimizer=optimizer)
        else:
            w, loss = local_model.update_weights(
                model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))
        
        log[epoch][idx] = {}
        log[epoch][idx]['status'] = 'malicious' if poison_ratio > 0 else 'clean'
        log[epoch][idx]['loss'] = loss
        log[epoch][idx]['weights'] = w

    # defense
    clean_weights = []
    poison_weights = []
    attackers = []
    if args.defense != 'fedavg':
        if args.defense == 'krum':
            honest_client = krum(local_weights, len(local_weights), 2)
            clean_weights = [local_weights[i] for i in honest_client]
            attackers = [i for i in range(len(local_weights)) if i not in honest_client]
        elif args.defense == 'multi_krum':
            num_malicious = int(args.attackers * m)
            n = int(m * 0.6)
            honest_client = multi_krum(local_weights, len(local_weights), num_malicious, n)
            clean_weights = [local_weights[i] for i in honest_client]
            attackers = [i for i in range(len(local_weights)) if i not in honest_client]
        elif args.defense == 'ours':
            clean_states = clean_model.state_dict()
            attackers = detect_outliers_from_weights(clean_states, local_weights, num_layers=12)
            clean_weights = [local_weights[i] for i in range(len(local_weights)) if i not in attackers]
        elif args.defense == 'trimmed_mean':
            clean_weights = trimmed_mean(local_weights, trim_ratio=0.1)
        elif args.defense == 'bulyan':
            num_malicious = int(args.attackers * m)
            n = int(m * 0.6)
            clean_weights = bulyan(local_weights, len(local_weights), num_malicious)

    
        print(f"Attackers: {attackers}")
        log[epoch]['attackers'] = attackers
    else:
        clean_weights = local_weights
        
        
    # update global weights
    if args.defense == 'trimmed_mean' or args.defense == 'bulyan':
        global_weights = clean_weights
    elif len(clean_weights) != 0:
        global_weights = average_weights(clean_weights)
    else:
        global_weights = global_model.state_dict()

    
    global_model = load_params(global_model, global_weights)    
    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    
    log[epoch]['global']['status'] = 'malicious' if attacked else 'clean'   
    log[epoch]['global']['loss'] = loss_avg
    log[epoch]['global']['weights'] = global_weights
    
    print(f' \nAvg Training Stats after {epoch + 1} global rounds:')
    print(f'Training Loss : {np.mean(np.array(train_loss))}')
    # print('Train Accuracy: {:.2f}% \n'.format(100 * train_accuracy[-1]))
    test_acc, _ = test_inference(args, global_model, test_dataset)
    test_asr, _ = test_inference(args, global_model, attack_test_set)
    print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
    print("|---- Test ASR: {:.2f}%".format(100 * test_asr))
    test_acc_list.append(test_acc)
    test_asr_list.append(test_asr)

In [16]:
test_acc, _ = test_inference(args, global_model, test_dataset)
test_asr, _ = test_inference(args,global_model, attack_test_set)
print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
print("|---- Test ASR: {:.2f}%".format(100 * test_asr))
test_acc_list.append(test_acc)
test_asr_list.append(test_asr)

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

|---- Test ACC: 88.30%
|---- Test ASR: 8.78%


In [19]:
attack_test_set['sentence'][1]

'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker .  c'

In [17]:
for epoch in log.keys():
    print(f"Epoch {epoch}\n")
    weights = []
    for idx in log[epoch].keys():
        if idx == 'attackers':
            print(f"Attackers: {log[epoch][idx]}")
        elif idx != 'global':
            print(f"Client {idx}: {log[epoch][idx]['status']}")
        # weights.append(log[epoch][idx]['weights'])

Epoch 0

Client 18: clean
Client 1: malicious
Client 19: clean
Client 8: malicious
Client 10: clean
Attackers: [1, 2, 3, 4]
Epoch 1

Client 3: clean
Client 16: clean
Client 6: clean
Client 10: clean
Client 2: clean
Attackers: [1, 2, 3, 4]
Epoch 2

Client 12: clean
Client 4: clean
Client 18: clean
Client 0: malicious
Client 9: clean
Attackers: [1, 2, 3, 4]


In [27]:
def trimmed_mean(client_state_dicts, trim_ratio=0.1):
    """
    Apply Trimmed Mean to a list of client updates with LoRA parameters.
    :param client_state_dicts: List of state_dicts, where each state_dict contains LoRA parameters for a client.
    :param trim_ratio: Proportion of the extreme values to trim from each end.
    :return: Aggregated state_dict based on trimmed mean of client updates.
    """
    num_clients = len(client_state_dicts)
    trim_count = int(trim_ratio * num_clients)  # Number of clients to trim from each end
    
    # Extract LoRA parameters and initialize aggregated weights dictionary
    param_keys = client_state_dicts[0].keys()
    aggregated_weights = {}

    # Iterate over each parameter key in the client state_dicts
    for key in param_keys:
        # Stack weights for the current parameter from all clients
        param_values = np.array([client[key].cpu().numpy() for client in client_state_dicts])

        # Sort and trim the parameter values across clients
        sorted_values = np.sort(param_values, axis=0)
        trimmed_values = sorted_values[trim_count:num_clients - trim_count]  # Trim top and bottom values
        
        # Calculate mean of trimmed values
        trimmed_mean = np.mean(trimmed_values, axis=0)
        
        # Store the trimmed mean in the aggregated weights dictionary
        aggregated_weights[key] = torch.tensor(trimmed_mean).to(client_state_dicts[0][key].device)

    return aggregated_weights

def bulyan(client_state_dicts, num_clients, num_byzantine_clients, trim_ratio=0.1):
    """
    Apply Bulyan aggregation to a list of client updates.
    :param client_state_dicts: List of state_dicts with LoRA parameters for each client.
    :param num_clients: Total number of clients.
    :param num_byzantine_clients: Number of suspected Byzantine clients.
    :return: Aggregated update based on Bulyan's robust aggregation.
    """
    multi_krum_set = multi_krum(client_state_dicts, num_clients, num_byzantine_clients, n=num_clients - 2 * num_byzantine_clients)

    selected_updates = [client_state_dicts[i] for i in multi_krum_set]

    return trimmed_mean(selected_updates, trim_ratio=trim_ratio)

trimmed_mean_weights = trimmed_mean(weights, trim_ratio=0.1)
bulyan_weights = bulyan(weights, len(weights), 2)

In [28]:
base_model = copy.deepcopy(clean_model).to(device)  
base_model = load_params(base_model, bulyan_weights)

test_acc, _ = test_inference(args, base_model, test_dataset)
test_asr, _ = test_inference(args, base_model, attack_test_set)
print("|---- Test ACC: {:.2f}%".format(100 * test_acc))
print("|---- Test ASR: {:.2f}%".format(100 * test_asr))

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

|---- Test ACC: 86.47%
|---- Test ASR: 10.59%


'bottom-rung new jack city '